# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [2]:
import pandas as pd

def clean_insurance_data(df):
    # 1. Clean column names
    df.columns = [col.replace(" ", "_") for col in df.columns]
    df.rename(columns={"ST": "state"}, inplace=True)
    df.columns = [col.lower() for col in df.columns]

    # 2. Standardize gender values
    df['gender'] = df['gender'].replace({
        'F': 'F', 'Femal': 'F', 'Female': 'F', 'female': 'F', 'f': 'F',
        'M': 'M', 'Male': 'M', 'male': 'M', 'm': 'M'
    })

    # 3. Standardize state values
    df['state'] = df['state'].replace({
        'Arizona': 'AZ',
        'Washington': 'WA',
        'Nevada': 'NE',
        'California': 'Cali',
        'Oregon': 'OR'
    })

    # 4. Standardize education values
    df['education'] = df['education'].replace({
        'Bachelors': 'Bachelor'
    })

    # 5. Group similar vehicle classes into "Luxury"
    df['vehicle_class'] = df['vehicle_class'].replace({
        'Sports Car': 'Luxury',
        'Luxury SUV': 'Luxury',
        'Luxury Car': 'Luxury'
    })

    # 6. Clean and convert customer_lifetime_value
    df['customer_lifetime_value'] = df['customer_lifetime_value'].astype(str).str.replace('%', '', regex=False)
    df['customer_lifetime_value'] = pd.to_numeric(df['customer_lifetime_value'], errors='coerce')

    # 7. Clean and convert number_of_open_complaints
    df['number_of_open_complaints'] = df['number_of_open_complaints'].apply(
        lambda x: int(str(x).split('/')[1]) if '/' in str(x) else pd.to_numeric(x, errors='coerce')
    )
    df['number_of_open_complaints'] = pd.to_numeric(df['number_of_open_complaints'], errors='coerce')

    # 8. Handle missing values
    numeric_cols = ['customer_lifetime_value', 'income', 'monthly_premium_auto',
                    'number_of_open_complaints', 'total_claim_amount']
    categorical_cols = ['customer', 'state', 'gender', 'education', 'policy_type', 'vehicle_class']

    # Приведение категориальных признаков к строковому типу
    for col in categorical_cols:
        df[col] = df[col].astype(str)

    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())

    for col in categorical_cols:
        mode = df[col].mode()
        if not mode.empty:
            df[col] = df[col].fillna(mode.iloc[0])

    # 9. Convert numeric columns to integers
    for col in numeric_cols:
        df[col] = df[col].astype(int)

    # 10. Drop duplicates and reset index
    df = df.drop_duplicates(keep='first').reset_index(drop=True)

    return df



In [3]:
clean_df = clean_insurance_data(df)
clean_df.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,0,DK49336,AZ,4809,No,Basic,College,2/18/11,Employed,M,...,0,9,Corporate Auto,Corporate L3,Offer3,Agent,292,Four-Door Car,Medsize,NaN
1,1,KX64629,Cali,2228,No,Basic,College,1/18/11,Unemployed,F,...,0,1,Personal Auto,Personal L3,Offer4,Call Center,744,Four-Door Car,Medsize,NaN
2,2,LZ68649,WA,14947,No,Basic,Bachelor,2/10/11,Employed,M,...,0,2,Personal Auto,Personal L3,Offer3,Call Center,480,SUV,Medsize,A
3,3,XL78013,OR,22332,Yes,Extended,College,1/11/11,Employed,M,...,0,2,Corporate Auto,Corporate L3,Offer2,Branch,484,Four-Door Car,Medsize,A
4,4,QA50777,OR,9025,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,0,7,Personal Auto,Personal L2,Offer1,Branch,707,Four-Door Car,Medsize,NaN


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [4]:
df_1 = df[(df['response'] == 'Yes') & (df['total_claim_amount'] > 1000)]
df_1.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,189,OK31456,Cali,11009,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358,Luxury,Medsize,NaN
236,236,YJ16163,OR,11009,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0,1,Special Auto,Special L3,Offer2,Agent,1358,Luxury,Medsize,A
419,419,GW43195,OR,25807,Yes,Extended,College,2/13/11,Employed,F,...,1,2,Personal Auto,Personal L2,Offer1,Branch,1027,Luxury,Small,A
442,442,IP94270,AZ,13736,Yes,Premium,Master,2/13/11,Disabled,F,...,0,8,Personal Auto,Personal L2,Offer1,Web,1261,SUV,Medsize,A
587,587,FJ28407,Cali,5619,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0,1,Personal Auto,Personal L1,Offer2,Web,1027,SUV,Medsize,A


In [5]:
df_2 = df[(df['response'] == 'Yes')]
df_2.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
3,3,XL78013,OR,22332,Yes,Extended,College,1/11/11,Employed,M,...,0,2,Corporate Auto,Corporate L3,Offer2,Branch,484,Four-Door Car,Medsize,A
8,8,FM55990,Cali,5989,Yes,Premium,College,1/19/11,Employed,M,...,0,1,Personal Auto,Personal L1,Offer2,Branch,739,Luxury,Medsize,NaN
15,15,CW49887,Cali,4626,Yes,Basic,Master,1/16/11,Employed,F,...,0,1,Special Auto,Special L1,Offer2,Branch,547,SUV,Medsize,NaN
19,19,NJ54277,Cali,3746,Yes,Extended,College,2/26/11,Employed,F,...,1,1,Personal Auto,Personal L2,Offer2,Call Center,19,Two-Door Car,Large,A
27,27,MQ68407,OR,4376,Yes,Premium,Bachelor,2/28/11,Employed,F,...,0,1,Personal Auto,Personal L3,Offer2,Agent,60,Four-Door Car,Medsize,NaN


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [6]:
pivot_table = df_2.pivot_table(
    values='total_claim_amount',
    index=['gender','policy_type'],
    aggfunc='mean')

print(pivot_table.sort_values(by='total_claim_amount', ascending=False))

                       total_claim_amount
gender policy_type                       
M      Personal Auto           456.576493
F      Special Auto            452.857143
       Personal Auto           452.498148
       Corporate Auto          433.295858
M      Special Auto            429.125000
       Corporate Auto          408.123377


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [15]:
pivot_table_2 = clean_df.pivot_table(
    values='customer',
    index='state',
    aggfunc='count')
filtered = pivot_table_2[pivot_table_2['customer'] > 500]


print(filtered.sort_values(by='customer', ascending=False))

       customer
state          
Cali       3552
OR         2909
AZ         1937
NE          993
WA          888
nan         631


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [16]:
clv_stats = clean_df.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['min', 'median', 'max'])


print(clv_stats)

                              min  median    max
education            gender                     
Bachelor             F       1904  5640.0  73225
                     M       1898  5548.0  67907
College              F       1898  5623.0  61850
                     M       1918  6005.0  61134
Doctor               F       2395  5332.0  44856
                     M       2267  5577.0  32677
High School or Below F       2144  6039.5  55277
                     M       1940  6286.5  83325
Master               F       2417  5729.0  51016
                     M       2272  5578.5  50568


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here